In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

/home/hadoop/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
loan_data = pd.read_csv("../data/train_data.csv")

loan_data.index = loan_data.userid
loan_data.drop(['userid'],axis=1,inplace=True)
loan_data.head()

,account#0,account#1,account#2,account#3,account#4,edu#0,edu#1,edu#2,edu#3,edu#4,...,prepare_amount#10,prepare_amount#11,prepare_amount#12,prepare_amount#13,prepare_amount#14,prepare_amount#15,prepare_amount#16,prepare_amount#17,prepare_amount#18,prepare_amount#19
userid,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
# overdue_train，这是我们模型所要拟合的目标
target = pd.read_csv('../../pcredit/train/overdue_train.txt',
                         header = None)
target.columns = ['userid', 'label']
target.index = target['userid']
target.drop('userid',
            axis = 1,
            inplace = True)
target.head()

,label
userid,
1,0
2,0
3,0
4,1
5,0


In [22]:
# 分开训练集、测试集
train = loan_data.iloc[0: 55596, :]
test = loan_data.iloc[55596:, :]
# 避免过拟合，采用交叉验证，验证集占训练集20%，固定随机种子（random_state)
train_X, test_X, train_y, test_y = train_test_split(train,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 0)

In [17]:
train_X.head()

,account#0,account#1,account#2,account#3,account#4,edu#0,edu#1,edu#2,edu#3,edu#4,...,prepare_amount#10,prepare_amount#11,prepare_amount#12,prepare_amount#13,prepare_amount#14,prepare_amount#15,prepare_amount#16,prepare_amount#17,prepare_amount#18,prepare_amount#19
userid,,,,,,,,,,,,,,,,,,,,,
18449,0,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22612,0,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21023,0,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32154,0,0,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
765,0,1,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train_y.head()

,label
userid,
18449,0
22612,0
21023,0
32154,0
765,0


In [30]:
lr_model = LogisticRegression(C = 1.0,
                              penalty = 'l2')
lr_model.fit(train_X, train_y)
# 给出交叉验证集的预测结果，评估准确率、召回率、F1值
pred_test = lr_model.predict(test_X)
print pred_test
print classification_report(test_y, pred_test)

[0 0 0 ..., 0 0 0]
             precision    recall  f1-score   support

          0       0.87      1.00      0.93      9703
          1       0.00      0.00      0.00      1417

avg / total       0.76      0.87      0.81     11120



In [34]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
clf = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300)
train_y = train_y['label']
clf = clf.fit(train_X, train_y)

scores = cross_val_score(clf, iris.data, iris.target)

pred_test = clf.predict(test_X)
print pred_test

[ 0.18821023  0.28799761  0.20062533 ...,  0.28799761  0.28799761
  0.28799761]


In [43]:
from pandas import DataFrame
pred_test = DataFrame(pred_test)
pred_test[pred_test[0] >= 0.5]

,0
548,0.504743
1965,0.600858
3519,1.000000
3595,0.600858
5298,0.594771
6549,0.594771
7640,0.750000
10412,0.750000
